In [ ]:
import numpy as np
from tqdm import tqdm

from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

import random

In [ ]:
path = '../dataset/'
dataset = "tiny-imagenet-200/"

In [ ]:
SIZE = 64 # this is the target size of the entire process

In [ ]:
# create 3 different generators: one for each folder

datagen = ImageDataGenerator(rescale=1. / 255) # rescaling input

train_gen = datagen.flow_from_directory(path+"download/"+dataset+"train/", 
                                          target_size=(SIZE, SIZE), 
                                          class_mode=None)
valid_gen = datagen.flow_from_directory(path+"download/"+dataset+"val/", 
                                          target_size=(SIZE, SIZE), 
                                          class_mode=None)
test_gen = datagen.flow_from_directory(path+"download/"+dataset+"test/", 
                                          target_size=(SIZE, SIZE), 
                                          class_mode=None)

In [ ]:
# Convert from RGB to Lab space
def convert_dataset(gen):
    X = []
    Y = []
    for i in tqdm(range(len(gen))):
        for img in gen[i]:
            lab = rgb2lab(img)
            X.append(lab[:,:,0])
            Y.append(lab[:,:,1:] / 128) # --> new range [-1,1]
    X = np.array(X)
    Y = np.array(Y)
    X = X.reshape(X.shape+(1,))
    print(X.shape, Y.shape)
    return X,Y

In [ ]:
X_train,Y_train = convert_dataset(train_gen)
X_valid,Y_valid = convert_dataset(valid_gen)
X_test,Y_test = convert_dataset(test_gen)

In [ ]:
np.save('./preprocessed/'+dataset+'x_train.npy', X_train)
np.save('./preprocessed/'+dataset+'y_train.npy', Y_train)

np.save('./preprocessed/'+dataset+'x_valid.npy', X_valid)
np.save('./preprocessed/'+dataset+'y_valid.npy', Y_valid)

np.save('./preprocessed/'+dataset+'x_test.npy', X_test)
np.save('./preprocessed/'+dataset+'y_test.npy', Y_test)